##Data Anonymization

Due to privacy concerns, several columns need to be dropped or recoded.
For example, information identifying students and outside adults (mentors, coaches, etc.) will be dropped.
A column that provides a unique identifier for each student will be used to create a new randomly generated identifier for each student.
Information identifying assigned teacher by name or email address will also be dropped, although a teacher ID column will be retained.

In [ ]:
import numpy as np
import pandas as pd
import random
import csv

In [ ]:
# read in source data Excel file
enr = pd.read_excel('ALVS_Enrollments_SY14-16.xlsx')

In [ ]:
# The 'CustomerID' column has a unique identifier for each student.
# Will use that column to create my own unique identifier and then export the references.
# Then create a column with the uniqute identifier and drop the original

student_id_list = enr['CustomerID'].unique()

student_id_dict = {}
fake_ids = range(1, len(student_id_list) + 1)
random.seed(1242)
random.shuffle(fake_ids)
i = 0
for id in student_id_list:
    student_id_dict[id] = fake_ids[i]
    i += 1

# Verify there's a new ID for each student
print "DF enr's shape: {}".format(enr.shape)
print "Number of values in customer ID column: {}".format(enr['CustomerID'].size)
print "Number of UNIQUE values in customer ID column: {}".format(len(enr['CustomerID'].unique()))
print "Number of ID's captured in Customer ID list: {}".format(len(student_id_list))
print "Number of oldID-newID pairs in dictionary: {}".format(len(student_id_dict))

In [ ]:
# Output CSV file with old ID and new ID
ID_file_header = ['old_ID','new_ID']
with open('oldID_newID_dict.csv', 'wb') as csvfile:
    filewriter = csv.writer(csvfile)
    filewriter.writerow(ID_file_header)
    for k, v in student_id_dict.iteritems():
        filewriter.writerow((k, v))

In [ ]:
# Create new ID column with the new student ID
enr['Anon_Student_ID'] = enr.CustomerID.map(lambda x: student_id_dict[x])

In [ ]:
privacy_cols_to_drop = ['CustomerName','CustomerID','ImportOrgID','ImportUserID','FirstName','MiddleName','LastName','LogonID','Email', 'MessageCenterID','StudentPhone','TeacherName','TeacherEmail','MentorCustomerID', 'MentorName', 'MentorEmail','SCCustomerID','SCName','SCEmail','CoachEmails']

In [ ]:
enr.drop(privacy_cols_to_drop, inplace=True, axis=1)

In [ ]:
# Verify each expected column was dropped
for x in privacy_cols_to_drop:
    if x in list(enr.columns):
        print x+","+"still there"
    else:
        print x+","+"successfully deleted"

In [ ]:
# Examine remaining columns
list(enr.columns)

In [ ]:
# export anonymized dataframe
# index=False because will use one of the columns as the index when re-importing
writer = pd.ExcelWriter("ALVS_Enrollments_SY14-16_Anon.xlsx")
enr.to_excel(writer,index=False,header=True,sheet_name="Enrollments_SY14-16")